In [6]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline


# initialize variables / model-parameters

W = tf.Variable(tf.zeros([5,1]), name="weights")
b = tf.Variable(0.0, name="bias")

In [7]:
# utility functions

def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.dirname('__file__') + "/" + file_name])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)
    
    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)
    
    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)

In [8]:
# read input data

def inputs():
    passenger_id,survived,pclass,name,sex,age,sib_sp,parch,ticket,fare,cabin,embarked = \
        read_csv(100, "data\titanic_training_data.csv", [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]] )

        
    # convert categorical data to 1-0
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))
    
    gender = tf.to_float(tf.equal(sex, ["female"]))
    
    # combine all input into one matrix with a sample per line and a feature per column
    features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100,1])
    
    return features, survived

In [13]:
# former inference is now used for combining inputs
def combine_inputs(X):
    return tf.matmul(X, W) + b

def inference(X):
    # compute the inference model over data X and return result
    return tf.matmul(X,W) + b
    
def loss(X,Y):
    # compute the loss (error) over training data X and expected outputs Y
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(combine_inputs(X), Y))
    return loss

def train(total_loss):
    # train/adjust model parameters according to total loss
    
    # use gradient descent
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    
def evaluate(sess, X, Y):
    # evaluate the resulting trained model
    
    predicted = tf.cast(inference(X) > 0.5, tf.float32)
    print sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32)))

In [ ]:
# launch the graph in a session, setup boilerplate

saver = tf.train.Saver()

with tf.Session() as sess:
    
    tf.initialize_all_variables().run()
    
    X, Y = inputs()
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)
    
    print(sess.run([total_loss]))
    print(sess.run([train_op]))

loss: Tensor("Mean_4:0", shape=(), dtype=float32)
